In [1]:
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import os as os

In [2]:
# (patient count* 15) check_path function will return total patient count 
inputs = np.zeros((108*15,256,512, 1))
outputs = np.zeros((108*15,256,512 , 9))

In [6]:
dic_count = 0
ann_count = 0
#set path 
folder_path = os.getcwd()+"/Incoming_Annotations"

for folders in sorted(os.listdir(folder_path)):
    image_path = os.path.join(folder_path, folders)
    if(dcm_to_np(image_path)):
        continue
    else:
        break
for files in sorted(os.listdir(folder_path)):
    image_path = os.path.join(folder_path,files)
    segmented_path = os.path.join(image_path, "NRRD","")
    for entry in os.listdir(segmented_path):
        segmented_path += entry
        segmentation = sitk.ReadImage(segmented_path)
        nrrd_to_np(segmentation)

15
30
45
60
75
90
105
120
135
150
165
180
195
210
225
240
255
270
285
300
315
330
345
360
375
390
405
420
435
450
465
480
495
510
525
540
555
570
585
600
615


TypeError: Can't convert 'Image' object to str implicitly

In [4]:
def dcm_to_np(path):
    global dic_count
    slice_filenames = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(path)
    try:
        image = sitk.ReadImage(slice_filenames)
        for x in range(15):
            current_slice = image[:, :, x]
            current_array = sitk.GetArrayFromImage(current_slice)
            inputs[dic_count]=current_array[:,:,np.newaxis]
            dic_count+=1
        return 1
    except:
        print("ERROR: " + path)
        return 0

In [5]:
def nrrd_to_np(path):
    global ann_count
    try:
        seg = sitk.GetArrayFromImage(path)
        for p in range(15):
            current_seg = seg[p]
            new_seg = np.zeros([256,512,9])
            new_seg[0][0][8]
            for y,a in enumerate(current_seg):
                for x,b in enumerate(a):
                    if b>0:
                        new_seg[y][x][b-1]=1
            threshold, upper, lower = 0.5, 1, 0
            new_seg[new_seg>threshold] = upper
            new_seg[new_seg<=threshold] = lower

            #new_seg has size of 256x512x9 
            outputs[ann_count]=new_seg
            ann_count+=1
        print(ann_count)
    except:
        print("ERROR: " + path)

In [10]:
def check_path(path):
    count=0
    for files in os.listdir(path):
        image_path = os.path.join(path,files)
        #print(image_path)
        count+=1
        #segmented_path=os.path.join(image_path,"NRRD")
    print(count)

In [ ]:
#Unet Structure Code Copy and Pasted from Github
import numpy as np
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x
def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [ ]:
X_train, y_train, X_test, y_test = train_test_split(inputs, outputs, test_size=0.2)


model = get_unet(Input([512, 256, 9]))
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=["dice_coef", "meanIOU", "binary_cross_entropy"])
results = model.fit(x=X_train, y=y_train, validation_data=(X_test, y_test), epochs=20, batch_size=8)